In [1]:
from pycbc import distributions
from pycbc.waveform import get_td_waveform
import matplotlib.pyplot as plt
import numpy as np
import pylab
from tqdm.notebook import tqdm
from gwpy.timeseries import TimeSeries
import pandas as pd
import os
import csv

In [6]:
print("Generated Binary Mass Distributions for BBH")
bbh_two_mass_distributions = distributions.Uniform(mass1=(10, 50),
                                               mass2=(10, 50))

bbh_two_mass_samples = bbh_two_mass_distributions.rvs(size=5000)

print("Generated Binary Mass Distributions for BNS")
bns_two_mass_distributions = distributions.Uniform(mass1=(1, 2),
                                               mass2=(1, 2))

bns_two_mass_samples = bns_two_mass_distributions.rvs(size=5000)

Generated Binary Mass Distributions for BBH
Generated Binary Mass Distributions for BNS


In [8]:
start_times_bbh = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5]
start_times_bns = [0, 0.5, 1, 1.5, 2, 2.5, 3]

for i in tqdm(range(len(bbh_two_mass_samples))):

    hp1, hc1 = get_td_waveform(approximant="SEOBNRv2",                                
                         mass1=bbh_two_mass_samples[i][0],
                         mass2=bbh_two_mass_samples[i][1],
                         delta_t=1.0/4096,
                         f_lower=40)


    hp2, hc2 = get_td_waveform(approximant="IMRPhenomPv2_NRTidal", 
                         mass1=bns_two_mass_samples[i][0],
                         mass2=bns_two_mass_samples[i][1],
                         delta_t=1.0/4096,
                         f_lower=40)
    

    bbh_signal = TimeSeries.from_pycbc(hp1)
    st1 = np.random.randint(0, 8)
    bbh_signal.t0 = start_times_bbh[st1]
    bbh_signal = (bbh_signal/(max(bbh_signal.max(), np.abs(bbh_signal.min()))))*0.2


    # Extract the last 1 sec from the BNS signal
    t = hp2.get_end_time()
    hp2 = hp2.time_slice(t-1, t)

    bns_signal = TimeSeries.from_pycbc(hp2)
    st2 = np.random.randint(0, 7)
    bns_signal.t0 = start_times_bns[st2]
    bns_signal = bns_signal.taper()
    bns_signal = (bns_signal/(max(bns_signal.max(), np.abs(bns_signal.min()))))*0.2

    noise = TimeSeries(np.random.normal(scale=.1, size=16384), sample_rate=4096)


    data1 = noise.inject(bbh_signal)
    data2 = noise.inject(bns_signal)

    data1 *= 1e-17
    data2 *= 1e-17
    noise *= 1e-17
    
    data1.write("data/bbh_merged/bbh_merged_noise_signal_"+str(i)+".txt")
    data2.write("data/bns_merged/bns_merged_noise_signal_"+str(i)+".txt")
    noise.write("data/noise/noise_"+str(i)+".txt")


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# MERGING BBH NOISE + SIgnal Templates into single file

path = "data/bbh_merged/"
files= os.listdir(path)
f = open('data/Final_BBH_Merged_Noise_Signal.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/5000 [00:00<?, ?it/s]

In [5]:
# MERGING BNS NOISE + SIgnal Templates into single file

path = "data/bns_merged/"
files= os.listdir(path)
f = open('data/Final_BNS_Merged_Noise_Signal.csv', 'w')
cw = csv.writer(f)

for i in tqdm(files):
    df = pd.read_csv(path+i,sep = ' ', header=None)
    c = df[:][1]
    cw.writerow(c)
f.close()

  0%|          | 0/5000 [00:00<?, ?it/s]

In [2]:
# MERGING NOISE Templates into single file

path_1 = "data/noise/"
files_1= os.listdir(path_1)
f1 = open('data/Final_Merged_Noise_Reduced_No_Abs.csv', 'w')
cw_1 = csv.writer(f1)

for i in tqdm(files_1):
    #print(files)
    df = pd.read_csv(path_1+i,sep = ' ', header=None)
    c = df[:][1]
    cw_1.writerow(c)
f1.close()

  0%|          | 0/5000 [00:00<?, ?it/s]

In [3]:
# # Alternate
# start_times_bbh = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5]
# start_times_bns = [0, 0.5, 1, 1.5, 2, 2.5, 3]

# for i in tqdm(range(5000)):

#     bbh_signal = TimeSeries.read("data/bbh_signal/bbh_merged_signal_"+str(i)+".txt")
#     st1 = np.random.randint(0, 8)
#     bbh_signal.t0 = start_times_bbh[st1]
#     bbh_signal = (bbh_signal/(max(bbh_signal.max(), np.abs(bbh_signal.min()))))*0.2


#     bns_signal = TimeSeries.read("data/bns_signal/bns_signal_"+str(i)+".txt")
#     st2 = np.random.randint(0, 7)
#     bns_signal.t0 = start_times_bns[st2]
#     bns_signal = bns_signal.taper()
#     bns_signal = (bns_signal/(max(bns_signal.max(), np.abs(bns_signal.min()))))*0.2

#     noise = TimeSeries(np.random.normal(scale=.1, size=16384), sample_rate=4096)


#     data1 = noise.inject(bbh_signal)
#     data2 = noise.inject(bns_signal)

#     data1 *= 1e-17
#     data2 *= 1e-17
#     noise *= 1e-17
    
#     data1.write("data/bbh_merged/bbh_merged_noise_signal_"+str(i)+".txt")
#     data2.write("data/bns_merged/bns_merged_noise_signal_"+str(i)+".txt")
#     noise.write("data/noise/noise_"+str(i)+".txt")

  0%|          | 0/5000 [00:00<?, ?it/s]